In [82]:
# import dependencies 
# from selenium import webdriver
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import re
from urllib.request import urlopen
# import time
import pandas as pd
import numpy as np

In [83]:
executable_path = {'executable_path': 'C:\\Users\\Trisarah\\Desktop\chromedriver.exe'} 
browser= Browser('chrome', **executable_path, headless=False)
base_url = 'https://www.ulta.com'

In [84]:
# # set urls
ulta_urls = ['https://www.ulta.com/skin-care-cleansers?N=2794']
#               'https://www.ulta.com/skin-care-moisturizers?N=2796',
#               'https://www.ulta.com/skin-care-treatment-serums?N=27cs',
#               'https://www.ulta.com/skin-care-eye-treatments?N=270k',
#               'https://www.ulta.com/skin-care-suncare?N=27fe',
#               'https://www.ulta.com/makeup-face?N=26y3',
#               'https://www.ulta.com/makeup-eyes?N=26yd',
#               'https://www.ulta.com/makeup-lips?N=26yq',
#               'https://www.ulta.com/hair-shampoo-conditioner?N=27ih',
#               'https://www.ulta.com/hair-treatment?N=26xy',
#               'https://www.ulta.com/hair-styling-products?N=26xf',
#               'https://www.ulta.com/hair-color?N=26xs',
#               'https://www.ulta.com/bath-body-bath-shower?N=26uy',
#               'https://www.ulta.com/bath-body-body-moisturizers?N=26v3',
#               'https://www.ulta.com/bath-body-hand-foot-care?N=27ic',
# #               'https://www.ulta.com/bath-body-personal-care?N=27i3',
#               'https://www.ulta.com/bath-body-suncare?N=276b']

In [180]:
# create empty lists to append later
products = []
links = []

# loop through ulta urls
for url in ulta_urls:
    browser.visit(url)
    r = requests.get(url, timeout=5)
    
    # parse html source
    html = browser.html
    data = bs(html, 'html.parser')
    
    # get product link, title, and 
    prod_title = data.find_all('h4', attrs={'prod-title'})
    
    #loop through the links and generate unique url for each product
    for link in prod_title:
        find_href = link.findAll('a')
        
        # create empty dict
        prod_dict = {}
        
        # find product url, brand, description and url
        for a in find_href:
            href = a['href']
            prod_dict['url'] = base_url + a['href']
            link = base_url + a['href']
            links.append(link)
            prod_dict['id'] = prod_dict['url'].rsplit('=', 1)[+1] 
            prod_dict['descrip'] = re.findall(r"\/([\s\S]*?)\?", href)[0]
            prod_dict['brand'] = a.getText().split('\n\t\t\t\t')[1]
                 
#         append product dictionary to products list
            products.append(prod_dict)

In [181]:
##### THIS CELL WORKS ##### 

ingredients_dict = {}

urls = []
for link in links:
    urls.append(link)
    ingredients_dict['url'] = urls

for i in range(len(links)):
    try: 
        r = requests.get(url, timeout=5)
        html = urlopen(links[i]).read()
        soup = bs(html, 'html.parser')

        # get product information how to use, ingredients, and coupon info
        product_info = soup.findAll('div', attrs={'class': 'ProductDetail__ingredients'})

        # loop through product info to get ingredients 
        for ingredients in product_info:
            # will print out div with ingredients
            content = ingredients.find_all('div', {'class' : 'ProductDetail__productContent'})

            #### EVERYTHING ABOVE WORKS 

            for ing_list in content: 
                new_list = []

                ing_text = ing_list.getText()
                new_list.append(ing_text)
                ingredients_list.append(new_list)
                
                ingredients_dict['ingredients'] = ingredients_list
                
    except:
        ingredients_list.append('error url')
        print('could not find')

ValueError: arrays must all be same length

In [102]:
pd.DataFrame(products).head()

brand                                      descrip  \
0          Murad                  ahabha-exfoliating-cleanser   
1  Mario Badescu                    glycolic-foaming-cleanser   
2     Philosophy  purity-made-simple-one-step-facial-cleanser   
3  Mario Badescu                         enzyme-cleansing-gel   
4       Proactiv                     deep-cleansing-face-wash   

                   id                                                url  
0         prod2082342  https://www.ulta.com/ahabha-exfoliating-cleans...  
1   xlsImpprod6200753  https://www.ulta.com/glycolic-foaming-cleanser...  
2   xlsImpprod1490012  https://www.ulta.com/purity-made-simple-one-st...  
3   xlsImpprod6200755  https://www.ulta.com/enzyme-cleansing-gel?prod...  
4  xlsImpprod16621071  https://www.ulta.com/deep-cleansing-face-wash?...

In [144]:
len(ingredients_list)

372

In [145]:
len(products)

96

In [116]:
# Sample Result DataFrame
sample_result_df = pd.DataFrame(products)

In [118]:
toxic_chems = ['CI' , 'FD&C', 'D&C', 'P-Phenylenediamine','Formaldehyde','Paraben', 
               'Alkyl Parahydroxy Benzoates','Cyclotetrasiloxane', 'Octamethylcyclotetrasiloxane', 
               'Cylcopentasiloxane', 'Polydimethylsiloxan','Sodium Lauryl Sulfate', 'SLS', 
               'Sodium Laureth Sulfate', 'Sles', 'Sodium Lauryl Sulfoacetate', 'Sodium Lauroyl Isoethionate', 
               'Sodium Lauroyl Taurate', 'Sodium Cocoyl Isoethionate', 'Sodium Lauroyl Methyl Isoethionate', 
               'Sodium Lauroyl Sarcosinate', 'Disodium Laureth Sulfosuccinate','Benzophenone', 'Paba', 
               'Avobenzone', 'Homosalate', 'Ethoxycinnmate', 'Oxybenzone']

# For loop to create columns with default values of '0'
for chem in toxic_chems:
    sample_result_df[chem] = 0

In [119]:
sample_cols = sample_result_df.columns
sample_cols

Index(['brand', 'descrip', 'id', 'url', 'Sodium Chloride', 'Lead', 'CI',
       'FD&C', 'D&C', 'P-Phenylenediamine', 'Formaldehyde', 'Paraben',
       'Alkyl Parahydroxy Benzoates', 'Cyclotetrasiloxane',
       'Octamethylcyclotetrasiloxane', 'Cylcopentasiloxane',
       'Polydimethylsiloxan', 'Sodium Lauryl Sulfate', 'SLS',
       'Sodium Laureth Sulfate', 'Sles', 'Sodium Lauryl Sulfoacetate',
       'Sodium Lauroyl Isoethionate', 'Sodium Lauroyl Taurate',
       'Sodium Cocoyl Isoethionate', 'Sodium Lauroyl Methyl Isoethionate',
       'Sodium Lauroyl Sarcosinate', 'Disodium Laureth Sulfosuccinate',
       'Benzophenone', 'Paba', 'Avobenzone', 'Homosalate', 'Ethoxycinnmate',
       'Oxybenzone'],
      dtype='object')

In [122]:
sample_result_df.head()

brand                                      descrip  \
0          Murad                  ahabha-exfoliating-cleanser   
1  Mario Badescu                    glycolic-foaming-cleanser   
2     Philosophy  purity-made-simple-one-step-facial-cleanser   
3  Mario Badescu                         enzyme-cleansing-gel   
4       Proactiv                     deep-cleansing-face-wash   

                   id                                                url  \
0         prod2082342  https://www.ulta.com/ahabha-exfoliating-cleans...   
1   xlsImpprod6200753  https://www.ulta.com/glycolic-foaming-cleanser...   
2   xlsImpprod1490012  https://www.ulta.com/purity-made-simple-one-st...   
3   xlsImpprod6200755  https://www.ulta.com/enzyme-cleansing-gel?prod...   
4  xlsImpprod16621071  https://www.ulta.com/deep-cleansing-face-wash?...   

   Sodium Chloride  Lead  CI  FD&C  D&C  P-Phenylenediamine     ...      \
0                0     0   0     0    0                   0     ...       
1                0     0   0     0    0                   0     ...       
2                0     0   0     0    0                   0     ...       
3                0     0   0     0    0                   0     ...       
4                0     0   0     0    0                   0     ...       

   Sodium Cocoyl Isoethionate  Sodium Lauroyl Methyl Isoethionate  \
0                           0                                   0   
1                           0                                   0   
2                           0                                   0   
3                           0                                   0   
4                           0                                   0   

   Sodium Lauroyl Sarcosinate  Disodium Laureth Sulfosuccinate  Benzophenone  \
0                           0                                0             0   
1                           0                                0             0   
2                           0                                0             0   
3                           0                                0             0   
4                           0                                0             0   

   Paba  Avobenzone  Homosalate  Ethoxycinnmate  Oxybenzone  
0     0           0           0               0           0  
1     0           0           0               0           0  
2     0           0           0               0           0  
3     0           0           0               0           0  
4     0           0           0               0           0  

[5 rows x 34 columns]

In [ ]:
if 'brand' in sample_cols:
    print('Yup, brand is inside columns')

# Approach

- Use the 'in' keyword for columns